# **1. Preprocessing**

In this part we are going to import libraries and preprocess the data to have the dataset ready for my analisys

---



Import the required libraries

In [ ]:
import numpy as np
import pandas as pd
import glob
from skimage import io
import matplotlib.pyplot as plt
import skimage.color as skc

from sklearn import metrics
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV

from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC

import csv

Unzip the data

In [ ]:
!unzip testResized.zip
!unzip trainResized.zip

Convert the images to grayscale and put them in name order to match the csv

In [ ]:
xTrain = [io.imread(file, as_gray=True) for file in sorted(glob.glob("trainResized/*.Bmp"), key=lambda x: int(x [13:-4]))]
xTest = [io.imread(file, as_gray=True) for file in sorted(glob.glob("testResized/*.Bmp"), key=lambda x: int(x [12:-4]))]
xTrain = np.stack(xTrain)
xTest = np.stack(xTest)

I remove some images of the dataset randomly to lighten it in order to avoid excessively long execution times by setting a seed in order to know which are the corresponding labels to be eliminated

In [ ]:
seed = 42
rnd = np.random.default_rng(seed)
xTrain = rnd.choice(xTrain, size=3000)

I read the csv with Numpy taking only the data relating to the images that I have not deleted

In [ ]:
myData = np.genfromtxt("trainLabels.csv", delimiter=",", dtype="str", skip_header=1)
yTrain = myData[:,1]
rnd = np.random.default_rng(seed)
yTrain = rnd.choice(yTrain, size=3000)

I divide my training set into 2 sets:

*   training set: is a data set of examples used during the learning process and is used to fit the parameters
*   test set: A test data set is a data set that is independent of the training data set, but that follows the same probability distribution as the training data set

In [ ]:
xTrain, xVal, yTrain, yVal = train_test_split(xTrain, yTrain, test_size = 0.25, random_state = 8)
xTrain = xTrain.reshape((xTrain.shape[0],-1))
xVal = xVal.reshape((xVal.shape[0],-1))
xTest = xTest.reshape((xTest.shape[0],-1))



---


# **1.   KNN**

In this part I will use the KNN model to classify my data

---



I define a series of parameters that will be combined by cross validation to identify the best combination

In [ ]:
params = {'n_neighbors':[3, 5, 11, 25], 'p':[1, 2, 4], 'weights':['uniform', 'distance']}
knc = KNeighborsClassifier()
knn_clf = GridSearchCV(knc, param_grid = params, cv = 5)

In [ ]:
knn_clf.fit(xTrain, yTrain)

I run the predict on my test set and print the accuracy, the best score (best average accuracy on the training set) and the best combination of parameters

In [ ]:
yPredict = knn_clf.predict(xVal)
knn_acc_score = metrics.accuracy_score(yVal, yPredict)
print("KNN Accuracy Score is: {}".format(knn_acc_score))
print('Best average accuracy: {}'.format(knn_clf.best_score_)) # best average accuracy on the training set
print('Best parameters: {}'.format(knn_clf.best_params_))



---


# **2.   Random Forest Classifier**





In this part I will use the Random forest classifier to classify my data

---

I define a series of parameters that will be combined by cross validation to identify the best combination(sample=image)

In [ ]:
params = {'max_depth': [5, 7, 9, None], 'min_samples_split': [2, 5, 10], 'n_estimators': [10, 100, 200, 500]}
rf = RandomForestClassifier(criterion='entropy', max_features='auto') # max_features "auto" = sqrt(n_features) number of features resonable considering also execution time
rf_clf = GridSearchCV(rf, param_grid = params, cv = 5)

In [ ]:
rf_clf.fit(xTrain, yTrain)

I run the predict on my test set and print the accuracy, the best score (best average accuracy on the training set) and the best combination of parameters

In [ ]:
yPredict = rf_clf.predict(xVal)
rf_acc_score = metrics.accuracy_score(yVal, yPredict)
print("Random Forest Accuracy Score is: {}".format(rf_acc_score))
print('Best average accuracy: {}'.format(rf_clf.best_score_))
print('Best parameters: {}'.format(rf_clf.best_params_))

Random Forest Accuracy Score is: 0.592
Best score: 0.5444444444444445
Best parameters: {'max_depth': None, 'min_samples_split': 2, 'n_estimators': 500}




---


# **3.   Support Vector Classificator**

In this part I will use the Support Vector Classification to classify my data


---



Balance complexity function with data fit

I define a series of parameters that will be combined by cross validation to identify the best combination

In [ ]:
params = {'kernel':['linear', 'poly', 'rbf'], 'C':[0.001, 0.01, 0.1, 1, 10, 100], 'gamma':['scale','auto']} #C regularization parameter
svc = SVC()
svc_clf = GridSearchCV(svc, param_grid = params, cv = 5, verbose=2)

In [ ]:
svc_clf.fit(xTrain, yTrain)

I run the predict on my test set and print the accuracy, the best score (best average accuracy on the training set) and the best combination of parameters

In [ ]:
yPredict = svc_clf.predict(xVal)
svc_acc_score = metrics.accuracy_score(yVal, yPredict)
print("Support Vector Accuracy Score is: {}".format(svc_acc_score))
print('Best average accuracy: {}'.format(svc_clf.best_score_))
print('Best parameters: {}'.format(svc_clf.best_params_))

Support Vector Accuracy Score is: 0.43333333333333335
Best score: 0.3928888888888889
Best parameters: {'C': 100, 'gamma': 'auto', 'kernel': 'rbf'}




---


# **4.   Final Result**

Having the accuracy of the 3 algorithms and the best parameters, I select the algorithm and the parameters that define the highest accuracy and perform the predict with the "real" test set

In [ ]:
rf = RandomForestClassifier(criterion='entropy', max_features='auto', max_depth=None, min_samples_split=2, n_estimators=500, verbose=2)
rf.fit(xTrain,yTrain)

I get the results

In [ ]:
testPredictRF = rf.predict(xTest)

I create the csv file to send to solve the challenge and get the result

In [ ]:
result = []
for i in range(6284,12504):
  temp = (i,testPredictRF[i-6284])
  result.append(temp)

with open('SubmissionRF.csv','w') as out:
    csv_out=csv.writer(out)
    csv_out.writerow(['ID','Class'])
    for row in result:
        csv_out.writerow(row)

Score: 0.40297



---

